<a href="https://colab.research.google.com/github/CattoYT/ValorantCommsBot/blob/rewrite/yolo_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask
!pip install ultralytics pillow
!pip install pyngrok

#from google.colab import drive
#drive.mount("/content/drive")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [14]:
import asyncio
import ultralytics
import numpy as np
from PIL import Image
import io
import os
from flask import Flask, request, jsonify
from pyngrok import conf, ngrok

from google.colab import userdata

conf.get_default().auth_token = userdata.get('NGROKToken')
app = Flask(__name__)

public_url = ngrok.connect("80", domain="cheaply-caring-pup.ngrok-free.app").public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, "80"))

app.config["BASE_URL"] = public_url

def overlayCensor(img):
        img_np = np.array(img)
        height, width, _ = img_np.shape
        box_coords = (width - 70, 93, width - 30, 400)  # (x_min, y_min, x_max, y_max)

        # Set the specified area to black
        img_np[box_coords[1]:box_coords[3], box_coords[0]:box_coords[2]] = [0, 0, 0]
        img = Image.fromarray(img_np)
        return img


@app.route("/inference", methods=['POST'])
def inference():
    imagedata = request.data

    image = Image.open(io.BytesIO(imagedata))

    results = model(overlayCensor(image), conf=0.70, device="0")

    detections = results[0].boxes
    class_ids = detections.cls.cpu().numpy() if detections.cls is not None else [] # thanks copilot
    detected = 0
    for class_id in class_ids:
      if class_id == 1.0: # check if the class id is actually an enemy
                    detected += 1



    # Return a response if needed
    return str(detected)
model = None
if __name__ == '__main__':
    if model == None:
      import ultralytics
      modelPath = "/content/drive/MyDrive/epoch270.pt"
      model = ultralytics.YOLO(modelPath)
      # converts the model to engine if it isnt already
      print(modelPath[:-3] + ".engine")
      if modelPath[-2:] == "pt":
          try:
              model = ultralytics.YOLO(modelPath[:-3] + ".engine")
          except:
              if input("Do you want to convert this model to "
                       "TensorRT for faster inference? (y/n): ").lower() == "y":
                  model.export(dynamic=True, format="engine")
                  os.remove(modelPath[:-3] + ".onnx")
                  model = ultralytics.YOLO(modelPath[:-3] + ".engine")
    app.run(host='0.0.0.0', port=80)

Exception in thread Thread-32 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


 * ngrok tunnel "https://cheaply-caring-pup.ngrok-free.app" -> "http://127.0.0.1:80"
/content/drive/MyDrive/epoch270.engine
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:80
 * Running on http://172.28.0.12:80
INFO:werkzeug:Press CTRL+C to quit


Loading /content/drive/MyDrive/epoch270.engine for TensorRT inference...

0: 640x640 1 enemy, 15.4ms
Speed: 3.0ms preprocess, 15.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [02/Aug/2024 10:45:20] "POST /inference HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [02/Aug/2024 10:47:04] "POST /inference HTTP/1.1" 200 -



0: 640x640 1 enemy, 15.5ms
Speed: 3.9ms preprocess, 15.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [02/Aug/2024 10:47:05] "POST /inference HTTP/1.1" 200 -



0: 640x640 1 enemy, 15.5ms
Speed: 4.2ms preprocess, 15.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [02/Aug/2024 10:47:06] "POST /inference HTTP/1.1" 200 -


In [ ]:
import os

async def yoloV8Detection(self):
      # This needs to run as fast as possible cuz valorant is fast asf
      import logging
      # Suppress logging output
      if self.model == None:
          modelPath = "Models/epoch270.pt"
          self.model = ultralytics.YOLO(modelPath)
          # converts the model to engine if it isnt already
          print(modelPath[:-3] + ".engine")
          if modelPath[-2:] == "pt":
              try:
                  self.model = ultralytics.YOLO(modelPath[:-3] + ".engine")
              except:
                  if input("Do you want to convert this model to "
                           "TensorRT for faster inference? (y/n): ").lower() == "y":
                      self.model.export(dynamic=True, format="engine")
                      os.remove(modelPath[:-3] + ".onnx")
                      self.model = ultralytics.YOLO(modelPath[:-3] + ".engine")
      cooldown = 0
      logging.getLogger('ultralytics').setLevel(logging.WARNING) #chatgpt'd because the documentation is kinda shit
          screenshot = detectors.capture_screenshot()
          results = self.model(self.overlayCensor(screenshot), conf=0.70, device="0")
          detections = results[0].boxes
          class_ids = detections.cls.cpu().numpy() if detections.cls is not None else [] # thanks copilot
          detected = 0
          for class_id in class_ids:
              if class_id == 1.0: # check if the class id is actually an enemy
                  detected += 1
          if detected >= 1:
              if time.time() - cooldown > 30:  # 30 seconds
                  cooldown = time.time()
                  spk.sayVoice("voices/mio/new-round/respawn - more_enemies_what_do_we_do.wav")

In [7]:
import sys
import glob
import serial

for i in serial_ports():
  print(i)

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result